# AIM
In this notebook we train our baseline model which has a similar architecture of Le-Net5.

## Setting Paths

In [3]:
import sys
from pathlib import Path
#Set root to be the main project folder
root = Path.cwd().parent
print(f"{root=}")

root=PosixPath('/home/gnacikm/Documents/GitHub/Melanoma')


In [4]:
data_path = Path(root/'data')
print(f"{data_path=}")
sav_models = Path(root/'saved_models')
print(f"{sav_models =}")

#Add location of py files to path so we can import
sys.path.insert(0,str(root))

data_path=PosixPath('/home/gnacikm/Documents/GitHub/Melanoma/data')
sav_models =PosixPath('/home/gnacikm/Documents/GitHub/Melanoma/saved_models')


## LIBRARIES / PACKAGES

In [6]:
from datetime import datetime
import tensorflow as tf

### Importing Models

In [7]:
from models.lenet import LeNet

### Importing Methods from Utils

In [8]:
from utils.tools import ImageGenerator, FlowFromDir

### Checking if GPU has been detected

In [10]:
device_name = tf.test.gpu_device_name()
if device_name == '':
    raise SystemError("GPU not found")
else:
    print(f"GPU {device_name} found")

GPU /device:GPU:0 found


## Processing Images

#### Setting paths

In [9]:
dir_with_zip = data_path
dir_with_data = f"{dir_with_zip}/DermMel" 
path_train = f"{dir_with_data}/train_sep/"
path_val = f"{dir_with_data}/valid/"
path_test = f"{dir_with_data}/test/"

#### Using ImageDataGenerator
See https://keras.io/api/preprocessing/image/

In [10]:
data_train = ImageGenerator()
data_val = ImageGenerator()
data_test = ImageGenerator()

In [11]:
train_generator = FlowFromDir(
    data_train,
    path_train,
    shuffle=True)
val_generator = FlowFromDir(
    data_val,
    path_val,
    shuffle=True)
test_generator = FlowFromDir(
    data_test,
    path_test,
    batch_size=1)

Found 10683 images belonging to 2 classes.
Found 3562 images belonging to 2 classes.
Found 3561 images belonging to 2 classes.


### Input and output shapes

In [10]:
input_shape = train_generator[0][0].shape[1:]
input_shape

(160, 160, 3)

In [11]:
num_of_classes = train_generator[0][1][0].shape[0]
num_of_classes 

2

### Setting model parameters

In [12]:
EPOCHS = 20
inputs = tf.keras.Input(input_shape, name="img")
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-04)

## Initiating the model

In [20]:
BaselineModel = LeNet(num_of_classes)
BaselineModel.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=["accuracy"]
)

### Callbacks

In [14]:
# Tensorboard https://www.tensorflow.org/tensorboard/scalars_and_keras
logdir = "../logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

2021-08-06 08:44:01.390224: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-06 08:44:01.390277: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-06 08:44:01.398224: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2021-08-06 08:44:01.398579: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2021-08-06 08:44:01.419041: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcupti.so
2021-08-06 08:44:01.522180: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2021-08-06 08:44:01.522439: I tensorflow/core/profiler/lib/pr

In [15]:
# ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    sav_models/"weights/lenet.h5", 
    monitor = "val_accuracy", 
    verbose = 2, 
    save_weights_only=True,
    save_best_only = True,
    mode = 'max'
)

### Fitting the model

In [18]:
history = BaselineModel.fit(train_generator, steps_per_epoch=train_generator.n//train_generator.batch_size, 
                    epochs=EPOCHS, 
                    validation_data=val_generator, 
                    validation_steps=val_generator.n//val_generator.batch_size,
                    callbacks=[tensorboard_callback, checkpoint])

2021-08-05 17:38:10.295988: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-08-05 17:38:11.805549: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-08-05 17:38:13.601702: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-05 17:38:13.601790: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2021-08-05 17:38:13.603576: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-08-05 17:38:13.603845: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2021-08-05 17:38:13.658118: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

Epoch 1/20
  3/667 [..............................] - ETA: 1:02 - loss: 0.7632 - accuracy: 0.4792 

2021-08-05 17:38:16.110211: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-05 17:38:16.110239: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-05 17:38:16.111064: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2021-08-05 17:38:16.129030: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-05 17:38:16.132419: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-08-05 17:38:16.134679: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-05 17:38:16.143181: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ../logs/scalars/20210805-173805/train/plug

667/667 [==============================] - 126s 187ms/step - loss: 0.6106 - accuracy: 0.6404 - val_loss: 0.5687 - val_accuracy: 0.6681

Epoch 00001: val_accuracy improved from -inf to 0.66807, saving model to /home/gnacikm/Documents/GitHub/Melanoma/saved_models/weights/lenet.h5
Epoch 2/20
667/667 [==============================] - 105s 157ms/step - loss: 0.5507 - accuracy: 0.6896 - val_loss: 0.5072 - val_accuracy: 0.6858

Epoch 00002: val_accuracy improved from 0.66807 to 0.68581, saving model to /home/gnacikm/Documents/GitHub/Melanoma/saved_models/weights/lenet.h5
Epoch 3/20
667/667 [==============================] - 106s 158ms/step - loss: 0.5140 - accuracy: 0.7295 - val_loss: 0.4914 - val_accuracy: 0.7801

Epoch 00003: val_accuracy improved from 0.68581 to 0.78012, saving model to /home/gnacikm/Documents/GitHub/Melanoma/saved_models/weights/lenet.h5
Epoch 4/20
667/667 [==============================] - 105s 157ms/step - loss: 0.4722 - accuracy: 0.7727 - val_loss: 0.4336 - val_accura

### Model Summary

In [21]:
BaselineModel.call(inputs)
BaselineModel.built = True 
BaselineModel.load_weights(sav_models/"weights/lenet.h5")
BaselineModel.summary()

Model: "le_net_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 158, 158, 32)      896       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 77, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 79, 79, 32)        0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 38, 38, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 92416)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 120)               11090040  
_________________________________________________________________
dense_4 (Dense)              (None, 84)                101

2021-08-06 08:46:23.342325: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 44359680 exceeds 10% of free system memory.


### TensorBoard Visualisation

In [19]:
%load_ext tensorboard
%tensorboard --logdir ../logs/scalars

/usr/lib/python3.8/subprocess.py:946: ResourceWarning: subprocess 56020 is still running
  _warn("subprocess %s is still running" % self.pid,


## Prediction accuracy on test dataset

In [23]:
test_generator.reset()
BaselineModel.evaluate(test_generator, steps=test_generator.n//test_generator.batch_size )

3561/3561 [==============================] - 30s 8ms/step - loss: 0.2679 - accuracy: 0.8863


[0.26792553067207336, 0.886267900466919]